In [2]:
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
from google.oauth2 import service_account
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/Users/Eis4Elephant/Desktop/key.json'

client = language.LanguageServiceClient()

In [46]:
#so that text is visble when displaying df 
pd.options.display.max_colwidth=1000

In [3]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
% matplotlib inline


# Load Tweet Data

In [4]:
#raw tweets
pre_election_data=pd.read_excel('../Load_Tweets/data/Pre_election.xlsx')
post_election_data=pd.read_excel('../Load_Tweets/data/Post-Election.xlsx')

In [5]:
pre_tweet_text_all=pre_election_data['text'].str.cat(sep=' ')

In [221]:
post_election_full=pd.read_csv('../Load_Tweets/data/post_election_sentiment.csv')
post_election_full=post_election_full.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [220]:
pre_election_sentiment=pd.read_csv('../Load_Tweets/data/pre_election_sentiment.csv')
pre_election_sentiment=pre_election_sentiment.drop((['Unnamed: 0']), axis=1)

In [13]:
Obama_tweets=pd.read_csv('../Load_Tweets/data/Obama_tweets.csv')
Obama_tweets=Obama_tweets.drop(['Unnamed: 0', 'ID', 'has_media', 'is_retweet', 'nbr_favorite', 'nbr_reply', 'nbr_retweet', 'url', 'user_id', 'medias', 'is_reply', 'usernameTweet'], axis=1 )

In [21]:
Obama_tweets['datetime']=pd.to_datetime(Obama_tweets['datetime'])

In [31]:
o_pre_election=Obama_tweets[Obama_tweets['datetime'] < '11-06-2012']
o_pre_election= o_pre_election[o_pre_election['datetime'] > '11-06-2011']

In [141]:
o_post_election=Obama_tweets[Obama_tweets['datetime'] > '11-06-2012']
o_post_election= o_post_election[o_post_election['datetime'] < '11-06-2013']



# Analyze Sentiment

In [156]:
analysis_df=pd.DataFrame(columns=['text', 'sentiment', 'magnitude'])

# The text to analyze

for tweet in o_post_election['text'][1248:]:
    
    text = tweet
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)


    # Detects the sentiment of the text
    sentiment = client.analyze_sentiment(document=document).document_sentiment
    d={'text':text, 'sentiment': sentiment.score, 'magnitude': sentiment.magnitude}
    analysis_df=analysis_df.append(d, ignore_index=True)
#     print('Text: {}'.format(text))
#     print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))

In [157]:
# analysis_df=analysis_df.append(election_500_1021)
# analysis_df=analysis_df.reset_index(drop=True)
len(analysis_df)

807

In [158]:
OPE=OPE.append(analysis_df)

In [159]:
OPE.describe()

,sentiment,magnitude
count,2055.000000,2055.000000
mean,0.064964,0.352652
std,0.228280,0.307092
min,-0.900000,0.000000
25%,0.000000,0.100000
50%,0.000000,0.300000
75%,0.200000,0.600000
max,0.900000,2.200000


In [164]:
OPE[OPE['sentiment'] <-0.85]

,text,sentiment,magnitude
8,RT if you agree: It's unacceptable for Republicans in Congress to cut vital services instead of closing tax loopholes for millionaires.,-0.9,0.9
198,"Republican op-ed explains, ""There is no longer any credible scientific debate about the basic facts"" http:// OFA.BO/aNuXrE #ActOnClimate",-0.9,0.9
565,"From @thinkprogress : ""49 Republicans Who Say Shutting Down The Government Over Obamacare Is A Big Mistake"" http:// OFA.BO/fHy5Qb",-0.9,0.9
677,President Obama: Let's work together to make government work better—instead of treating it as an enemy or purposefully making it work worse.,-0.9,0.9


In [161]:
# OPE.to_csv('../Load_Tweets/data/obama_post_election_sentiment', index=False)

In [166]:
o_pre_s=pd.read_csv('../Load_Tweets/data/obama_pre_election_sentiment')

In [167]:
o_post_s=pd.read_csv('../Load_Tweets/data/obama_post_election_sentiment')

In [209]:
o_pre_s[o_pre_s['sentiment'].astype('float') <(-0.35)].count()

text         205
sentiment    205
magnitude    205
dtype: int64

In [211]:
205/len(o_pre_s)

0.04313065432358511

In [260]:
o_post_s[o_post_s['sentiment'].astype('float') <(-0.35)].count()

text         65
sentiment    65
magnitude    65
dtype: int64

In [261]:
65/len(o_post_s)

0.031630170316301706

In [245]:
pre_election_sentiment[pre_election_sentiment['sentiment'].astype('float') <(-0.35)].count()

text         623
sentiment    623
magnitude    623
dtype: int64

In [248]:
623/len(pre_election_sentiment)

0.16106514994829368

In [247]:
post_election_full[post_election_full['sentiment'].astype('float') <(-0.35)].count()

magnitude    356
sentiment    356
text         356
dtype: int64

In [249]:
356/len(post_election_full)

0.16382880809940176

In [255]:
post_election_full[post_election_full['sentiment'].astype('float') >(0.35)].count()

magnitude    520
sentiment    520
text         520
dtype: int64

In [257]:
520/len(post_election_full)

0.2393005062126093

In [258]:
pre_election_sentiment[pre_election_sentiment['sentiment'].astype('float') >(0.35)].count()

text         594
sentiment    594
magnitude    594
dtype: int64

In [259]:
594/len(pre_election_sentiment)

0.15356773526370218

In [252]:
o_pre_s[o_pre_s['sentiment'].astype('float') >(0.35)].count()

text         645
sentiment    645
magnitude    645
dtype: int64

In [217]:
645/len(o_pre_s)

0.13570376604249948

In [253]:
o_post_s[o_post_s['sentiment'].astype('float') >(0.35)].count()

text         228
sentiment    228
magnitude    228
dtype: int64

In [218]:
228/len(o_post_s)

0.11094890510948906

In [14]:
# post_election_full=post_election1187.append(analysis_df)
# post_election_full=post_election_full.reset_index(drop=True)

In [18]:
# post_election_full=post_election_full.drop('Unnamed: 0', axis=1)

In [19]:
post_election_full.describe()

,magnitude,sentiment
count,2173.000000,2173.000000
mean,0.723930,0.050161
std,0.514323,0.403411
min,0.000000,-0.900000
25%,0.300000,-0.200000
50%,0.700000,0.100000
75%,1.000000,0.300000
max,3.000000,0.900000


In [23]:
pre_election_sentiment.describe()

,Unnamed: 0,sentiment,magnitude
count,3868.000000,3868.000000,3868.000000
mean,1933.500000,0.000801,0.654369
std,1116.739749,0.363419,0.527799
min,0.000000,-0.900000,0.000000
25%,966.750000,-0.200000,0.200000
50%,1933.500000,0.000000,0.600000
75%,2900.250000,0.200000,1.000000
max,3867.000000,0.900000,3.200000


In [267]:
post_election_full[post_election_full['sentiment'].astype('float') >(0.35)]

,magnitude,sentiment,text
1,0.9,0.4,Together we dream of a Korea that is free a peninsula that is safe and families that are reunited once again! https://t.co/9tsZRCC83j
9,1.7,0.5,Franklin such a great photo. HAPPY 99th BIRTHDAY to your father @BillyGraham! https://t.co/l0c7RvvOa1
14,0.8,0.4,Stock market hit yet another all-time record high yesterday. There is great confidence in the moves that my Administration....
15,1.3,0.4,Thank you to President Moon of South Korea for the beautiful welcoming ceremony. It will always be remembered. https://t.co/bMvJz1iV2a
16,1.2,0.6,It was my great honor to have lunch with our INCREDIBLE U.S. and ROK troops at Camp Humphreys in South Korea. 🇺🇸🇰🇷 https://t.co/5NJirz4iNM https://t.co/kNLyONhBCC
18,0.6,0.6,I have great confidence in King Salman and the Crown Prince of Saudi Arabia they know exactly what they are doing....
20,1.2,0.6,My visit to Japan and friendship with PM Abe will yield many benefits for our great Country. Massive military &amp; energy orders happening+++!
25,0.9,0.4,Heading into the 12 days with great negotiating strength because of our tremendous economy. https://t.co/4HnXkR3EhW
26,1.0,0.5,It was a true honor to be at Yokota Air Base with our GREAT @USForcesJapan! https://t.co/KEXnIkkQks
27,0.7,0.7,Playing golf with Prime Minister Abe and Hideki Matsuyama two wonderful people! https://t.co/vYLULe0o2K


In [28]:
post_election_full[post_election_full['sentiment'].astype('float') <(-0.85)].count()

magnitude    48
sentiment    48
text         48
dtype: int64

In [15]:
post_election_full.to_csv('../Load_Tweets/data/post_election_sentiment.csv')

In [99]:
analysis_df[analysis_df['sentiment'].astype('float') >(0.8)]

,text,sentiment,magnitude
24,Debate polls look great - thank you!#MAGA #AmericaFirst [URL],0.8,0.8
27,Thank you for all of the great comments on the debate last night. Very exciting!,0.8,1.7
39,RT @DanScavino: WE LOVE OUR DEPLORABLES!!!#TrumpTrain #Debates2016 [URL],0.8,0.8
45,"In my administration, EVERY American will be treated equally, protected equally, and honored equally #Debate #BigLeagueTruth",0.9,0.9
100,Certainly has been an interesting 24 hours!,0.8,0.8
161,".@mike_pence is doing a great job - so far, no contest!",0.9,0.9
170,"""@lainey34210: @realDonaldTrump Great opening Pence💕""",0.8,0.8
174,Here we go - Enjoy!,0.9,0.9
189,"Melania and I extend our warmest greetings to those observing Rosh Hashanah here in the United States, in Israel, and around the world.",0.8,0.8
247,Great debate poll numbers - I will be on @foxandfriends at 7:00 to discuss. Enjoy!,0.8,1.6


In [144]:
pre_election_full[pre_election_full['sentiment'].astype('float') <(-0.85)]

NameError: name 'pre_election_full' is not defined

In [92]:
analysis_df['sentiment']=analysis_df['sentiment'].astype('float')
analysis_df.describe()

,sentiment,magnitude
count,692.000000,692.000000
mean,0.048121,0.593786
std,0.368215,0.484915
min,-0.900000,0.000000
25%,-0.100000,0.200000
50%,0.000000,0.500000
75%,0.300000,0.900000
max,0.900000,2.500000


In [131]:
import six
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
def entity_sentiment_text(text):
    """Detects entity sentiment in the provided text."""

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    document = types.Document(
        content=text.encode('utf-8'),
        type=enums.Document.Type.PLAIN_TEXT)

    # Detect and send native Python encoding to receive correct word offsets.
    encoding = enums.EncodingType.UTF32
    if sys.maxunicode == 65535:
        encoding = enums.EncodingType.UTF16

    result = client.analyze_entity_sentiment(document, encoding)

    for entity in result.entities:
        print('Mentions: ')
        print(u'Name: "{}"'.format(entity.name))
        for mention in entity.mentions:
            print(u'  Begin Offset : {}'.format(mention.text.begin_offset))
            print(u'  Content : {}'.format(mention.text.content))
            print(u'  Magnitude : {}'.format(mention.sentiment.magnitude))
            print(u'  Sentiment : {}'.format(mention.sentiment.score))
            print(u'  Type : {}'.format(mention.type))
        print(u'Salience: {}'.format(entity.salience))
        print(u'Sentiment: {}\n'.format(entity.sentiment))
print (tweet_text_cleaned[0:1000])
entity_sentiment_text(tweet_text_cleaned[0:1000])



PAY TO PLAY POLITICS. #CrookedHillary Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system! Crooked Hillary Clinton likes to talk about the things she will do but she has been there for 30 years - why didn't she do them? Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again. Lets get out &amp;… Join me Thursday in Florida Ohio!West Palm Beach, FL at noon:[URL]Cincinnati, OH this 7:30pm:[URL] Wow, @CNN Town Hall questions were given to Crooked Hillary Clinton in advance of big debates against Bernie Sanders. Hillary CNN FRAUD! Thank you Texas! If you haven't registered to VOTE- today is your last day. Go to: get ou… VOTER REGISTRATION DEADLINES TODAY. You can register now at: and get out to… DON'T LET HER FOOL US AGAIN. Crooked's State Dept gave special attention to "Friends of Bill" after the Haiti Earthquake. Unbelievable! RT @DonaldJTrumpJr: 13 states have voter registration deadli